<a href="https://colab.research.google.com/github/flow4u/public/blob/master/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid-19

Data Source: https://github.com/CSSEGISandData/COVID-19
 
 

To run this notebook: **CTRL + F9**

## Import Modules & Data



In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import ipywidgets as widgets2
from google.colab import widgets
import plotly.express as px

In [0]:
data_sets = ('Confirmed', 'Recovered', 'Deaths')
base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-"

for set in data_sets:
  df_temp = pd.read_csv(base_url+set+'.csv')
  df_temp = df_temp.drop(['Lat', 'Long'],axis=1)
  del df_temp['Province/State']
  # not all date columns have the same format, some are mm/d/yy, others mm/dd/yy
  # this will convert first to date, then back to string
  for col in df_temp.columns:
    try:
      col1 = col[:len(col)-2]+'20'+col[-2:]
      col_new = datetime.strptime(col1,'%m/%d/%Y')
      col_new = datetime.strftime(col_new, '%Y-%m-%d')
      df_temp.rename(columns={col: col_new}, inplace=True)
    except:
      pass
  df_temp = df_temp.groupby(['Country/Region']).sum()
  df_temp['Status'] = set
  try:
    df = df.append(df_temp, sort=False)
  except:
    df=df_temp

df2 = df 
df_status = df2.groupby(['Status']).sum()

df = df.reindex(sorted(df.columns), axis=1)
df_plot = df_status.T.reset_index()  

## Filters

When a change is made, use **CTRL + F10**

In [0]:
europe = ['Austria', 'Belgium', 'Croatia', 'Finland', 'Ireland', 'France',
          'Germany','Italy', 'Norway', 'Romania','Greece', 'Spain', 'Sweden',
          'Switzerland', 'North Macedonia', 'Denmark', 'UK', 'Netherlands',
          'San Marino', 'Iceland', 'Luxembourg', 'Monaco', 'North Ireland',
          'Czech Republic', 'Andorra', 'Portugal', 'Gibraltar', 'Hungary',
          'Liechtenstein', 'Bosnia and Herzegovina', 'Slovenia','Poland',
          'Lithuania','Georgia','Estonia', 'Latvia', 'Russia', 'Belarus',
          'Armenia', 'Ukraine', 'Serbia', 'Slovakia', 'Republic of Ireland',
          'Vatican City', 'Bulgaria', 'Malta', 'Moldova', 'Albania', 'Cyprus']

asia = ['Cambodia', 'Mainland China', 'Hong Kong', 'India', 'Azerbaijan', 
        'Japan', 'Malaysia', 'Macau', 'Nepal', 'Philippines', 
        'Singapore', 'South Korea', 'Sri Lanka', 'Taiwan', 'Thailand',
        'Vietnam', 'Indonesia', 'Afghanistan', 'Bahrain','Kuwait','Oman',
        'Iran', 'Iraq','Pakistan', 'Israel','Jordan', 'Lebanon', 'Qatar',
        'Saudi Arabia', 'United Arab Emirates', 'Palestine', 'Bhutan',
        'Bangladesh', 'Maldives', 'Brunei']

north_america = ['US', 'Canada', 'Mexico', 'Martinique', 'St. Martin']

south_america = ['Brazil', 'Dominican Republic', 'Saint Barthelemy',
                 'Argentina', 'Chile', 'Ecuador', 'Faroe Islands',
                 'Colombia', 'Costa Rica', 'Peru', 'Togo', 'French Guiana',
                 'Paraguay']

africa = ['Nigeria', 'Morocco', 'Senegal', 'Algeria', 'Egypt', 'Tunisia',
          'South Africa', 'Cameroon']

oceania = ['Australia', 'New Zealand']

others = ['Others']

continents = {'europe': europe, 'asia': asia, 'north_america': north_america,
              'south_america': south_america,'africa': africa,
              'oceania': oceania, 'others':others}

all_countries=[]
all_countries.extend(others)
all_countries.extend(europe)
all_countries.extend(asia)
all_countries.extend(africa)
all_countries.extend(south_america)
all_countries.extend(north_america)
all_countries.extend(oceania)

missing_countries = 'Missing: '
for t in df.index.unique():
  if all_countries.count(t) == 0:
    missing_countries = missing_countries + ", '" + t + "'"


# View Graphs

In [11]:
#@title
if missing_countries != 'Missing: ':
  print(missing_countries)
  print('')
  print('The above countries must be added to the appropiate filters in the code')
  print('The code for the filters can be found above this output')

country_function_list={}
items=[] # continent, countries, buttons
items_all=[] # countries, buttons
countries_list = []
tb_continents = list(continents.keys())
tb_continent_tabs = widgets.TabBar(tb_continents)
tb_graph_tabs = ['Deaths, Existing, Confirmed, and Recovered + Deaths', 'Estimated Recovery Days', 'Estimated Mortality Rate']
tb_graph = widgets.TabBar(tb_graph_tabs)

layout=widgets2.Layout(#max_width='100%',
                          grid_template_columns='repeat(auto-fill,130px',
                       )

def replace_char(a):
  variable_name = a.replace(' ', '')
  variable_name = variable_name.replace('.', '')
  return variable_name

def button_toggled(value):
  update_graphs()

def button_creation(country):
  button_created = widgets2.ToggleButton(value=False, description=country, button_style='')
  variable_name = replace_char(country)
  globals()[variable_name] = button_created
  return button_created

def create_selection():
  for i, tab_content in enumerate(tb_continents):
    with tb_continent_tabs.output_to(i, select=False):
      items.append([])
      countries = sorted(continents[tab_content])
      items[i] = [button_creation(country) for country in countries]
      ui = widgets2.GridBox(items[i], layout=layout)
      make_countries_list()
      display(ui)

  for item in items:
    for subitem in item:
      items_all.append(subitem)
      btn = sub_item_processing(subitem)
      code_i = btn+'.observe(button_toggled, "value")'
      exec(code_i)


def create_graphs():
  countries_list = make_countries_list()
  process(countries_list)
  if (countries_list):
    global country_selection
    with tb_graph.output_to(0, select=False):
      title = 'COVID-19: Showing' + country_selection
      labels_axis = {'Status':'', 'value':'# of people', 'index':'date'}
      category_color = {'Confirmed':'grey', 'Deaths':'red', 'Existing':'orange', 'Recovered':'green','Recovered + Deaths':'blue'}
      use_log=False
      df_simple = df_plot.melt(id_vars='index')

      fig = px.line(df_simple, x='index', y='value',
                    color='Status',
                    color_discrete_map=category_color,
                    labels=labels_axis,
                    log_y=use_log, 
                    title=title)
      fig.show()

      
    with tb_graph.output_to(1, select=False):
      title = 'COVID-19: Showing' + country_selection
      labels_axis = {'RecoveryDays':'# of recovery days', 'index':'date'}
      df_simple = df_recovery_time
      fig = px.line(df_simple, x='index', y='RecoveryDays',
                    labels=labels_axis, 
                    title=title)
      fig.show()

    with tb_graph.output_to(2, select=False):
      title = 'COVID-19: Estimated Mortality Rate of' + country_selection
      labels_axis = {'variable':'', 'value':'%', 'index':'date'}
      category_color = {'Upper estimate':'red', 'Lower estimate':'orange'}

      fig = px.line(df_mortality_rate, x='index', y='value',
                    color='variable',
                    color_discrete_map=category_color,
                    labels=labels_axis, 
                    title=title)
      fig.show()



def update_graphs():
  countries_list = make_countries_list()
  for i, _ in enumerate(tb_graph_tabs):
    with tb_graph.output_to(i, select=False):
      tb_graph.clear_tab()
  create_graphs()

def sub_item_processing(a):
  b = replace_char(a.description)
  global country_function_list
  country_function_list[b] = a
  return b

def process(countries_list):
  if (countries_list):
    global country_selection
    country_selection = ''
    for t, c in enumerate(countries_list):
      if (t%8==0):
        country_selection += '\n' + c
      else:
        country_selection += ', ' + c

      # creating the different dataframes
    df2 = df[df.index.isin(countries_list)]
    df_status = df2.groupby(['Status']).sum()
    global df_plot
    df_plot = df_status.T.reset_index()
    df_plot['Existing'] = df_plot['Confirmed'] - df_plot['Deaths'] - df_plot['Recovered']
    df_plot['Recovered + Deaths'] = df_plot['Recovered'] +df_plot['Deaths']
    df_temp2 = df_status.T.reset_index()

    date_list=[]
    days_list=[]
    mortality_rate_list1=[]
    mortality_rate_list2=[]
    mortality_rate=[]
    value1_temp = -1
    value2_temp = -1
    value3_temp = -1
    for index, row in df_plot.iterrows():
      if row['Recovered'] or row['Deaths']> 0:
        value1_lookup = row['Recovered']
        value2_lookup = row['Confirmed']
        value3_lookup = row['Deaths']
        mortality_rate1 = value3_lookup / (value1_lookup+value3_lookup) * 100
        mortality_rate2 = value3_lookup / (value2_lookup) * 100
        if(value1_temp != value1_lookup or value2_temp != value2_lookup or value3_temp != value3_lookup):
          date1 = datetime.strptime(row['index'], '%Y-%m-%d')
          for index2, row2 in df_temp2.iterrows():
            if row2['Confirmed'] >= (value1_lookup+value3_lookup):
              
              date2 = datetime.strptime(row2['index'], '%Y-%m-%d')
              recovery_time = date1-date2
              recovery_time_date = datetime.strftime(date1, '%Y-%m-%d')
              recovery_days = recovery_time.days
              date_list.append(recovery_time_date)
              days_list.append(recovery_days)
              mortality_rate_list1.append(mortality_rate1)
              mortality_rate_list2.append(mortality_rate2)
              mortality_rate.append([mortality_rate1, mortality_rate2])
              break
          value1_temp = value1_lookup
          value2_temp = value2_lookup
          value3_temp = value3_lookup

    global df_recovery_time, df_mortality_rate
    df_recovery_time_temp = pd.DataFrame(days_list, index=date_list, columns=['RecoveryDays'])
    df_mortality_rate_temp = pd.DataFrame(mortality_rate, index=date_list, columns=['Upper estimate','Lower estimate'])
    df_recovery_time = df_recovery_time_temp.reset_index()
    df_mortality_rate_temp = df_mortality_rate_temp.reset_index()
    df_mortality_rate = df_mortality_rate_temp.melt(id_vars='index')



def make_countries_list():
  countries_list = []
  for item in items:
    for sub_item in item:
      if (sub_item.value == True):
        countries_list.append(sub_item.description)
  return countries_list

create_selection()
create_graphs()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>